# SIMULACIÓ BASE DE DADES CIRCUIT DE GAS AMB FUITA

## LLIBRERIES

In [ ]:
import numpy as np
import pandas as pd
import pandapipes as pp
import networkx as nx
from tqdm import tqdm
from scipy import stats
import logging
import matplotlib.pyplot as plt
import seaborn as sns

## CONFIGURACIÓ I PARÁMETRES

In [ ]:
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

np.random.seed(42)

PARAMS = {
    'prob_fuga': 0.8,
    'num_fugas': (1, 3),
    'tasa_fuga': (0.05, 0.1),
    'consumo_normal': (0.0001, 0.0003),
    'ruido_presion': (0.0001, 0.001),
    'prob_fallo_sensor': 0.02,
    'num_consumos': (4, 8),
    'prob_cierre_valvula': 0.03
}

## simulate_sensor_reading

In [ ]:
def simulate_sensor_reading(p_real, time_step):
    if np.random.rand() < PARAMS['prob_fallo_sensor']:
        return p_real
    sigma = np.random.uniform(*PARAMS['ruido_presion'])
    return p_real + np.random.normal(0, sigma)

## create_network

In [ ]:
def create_network(p_ext=1.5, t_ext=293.15, sim_id_for_name="default_sim"):
    net = pp.create_empty_network(fluid="lgas", name=f"sim_{sim_id_for_name}")
    nodes = list(range(100, 138)) 
    node_dict = {}

    for node_id in nodes:
        node_dict[node_id] = pp.create_junction(
            net,
            index=node_id,
            pn_bar=1.0, 
            tfluid_k=t_ext, 
            height_m=0,
            name=f"Node_{node_id}"
        )

    
    pp.create_ext_grid(net, junction=node_dict[100], p_bar=p_ext, t_k=t_ext, name="ExternalGrid_100")

    pipes_data = [ 
        (0, 100, 101, 216.1, 6), (1, 101, 102, 199.9, 6), (2, 102, 106, 98.1, 8),
        (3, 105, 106, 78.9, 3),  (4, 104, 105, 139.9, 3), (5, 101, 104, 92.0, 4),
        (6, 103, 104, 119.9, 3), (7, 100, 103, 81.1, 8),  (8, 100, 120, 135.9, 6),
        (9, 120, 121, 20.1, 4),  (10, 121, 134, 93.9, 4), (11, 122, 121, 75.9, 4),
        (12, 122, 123, 78.0, 6), (13, 122, 126, 153.0, 4), (14, 123, 124, 70.1, 6),
        (15, 124, 125, 63.1, 6), (16, 125, 126, 110.0, 3), (17, 126, 127, 60.0, 3),
        (18, 126, 129, 96.0, 4), (19, 134, 127, 96.9, 4), (20, 125, 128, 100.9, 4),
        (21, 128, 129, 73.2, 4), (22, 128, 131, 89.9, 3), (23, 129, 130, 67.1, 4),
        (24, 127, 130, 92.0, 4), (25, 130, 137, 95.1, 4), (26, 137, 133, 67.1, 3),
        (27, 132, 137, 98.1, 3), (28, 133, 135, 267.9, 3), (29, 135, 134, 60.0, 3),
        (30, 135, 136, 70.1, 3), (31, 103, 136, 49.8, 8), (32, 136, 107, 53.0, 8),
        (33, 107, 108, 223.1, 4), (34, 108, 109, 221.9, 4), (35, 106, 109, 93.0, 8),
        (36, 109, 118, 15.8, 6), (37, 118, 117, 111.9, 6), (38, 116, 117, 88.1, 6),
        (39, 117, 119, 84.1, 6), (40, 115, 116, 70.1, 6), (41, 113, 115, 150.0, 6),
        (42, 108, 113, 103.9, 4), (43, 110, 113, 252.1, 6), (44, 113, 114, 89.9, 4),
        (45, 107, 110, 118.9, 8), (46, 110, 111, 152.1, 12), (47, 111, 112, 56.1, 12),
        (48, 132, 131, 50.0, 8), (49, 114, 116, 30.0, 6), (50, 104, 108, 100.0, 4)
    ]

    for pipe_id, start, end, length, diameter in pipes_data:
        if start not in node_dict or end not in node_dict:
            logging.warning(f"Sim {net.name}: Skipping pipe {pipe_id} as node {start} or {end} not in node_dict.")
            continue
        pp.create_pipe_from_parameters(
            net,
            from_junction=node_dict[start],
            to_junction=node_dict[end],
            length_km=length/1000,
            diameter_m=diameter*0.0254,
            sections=10, 
            k_mm=0.05,   
            name=f"Pipe_{pipe_id}"
        )

    G_initial = nx.Graph()
    if not net.pipe.empty:
        G_initial.add_edges_from([(net.pipe.at[i, 'from_junction'], net.pipe.at[i, 'to_junction']) for i in net.pipe.index])

    source_node = net.ext_grid.junction.iloc[0]
    closed_valves_info = []

    if G_initial.nodes: 
        for pipe_idx in net.pipe.index.copy(): 
            if np.random.rand() < PARAMS['prob_cierre_valvula']:
                from_junc = net.pipe.at[pipe_idx, 'from_junction']
                to_junc = net.pipe.at[pipe_idx, 'to_junction']
                diameter_val = net.pipe.at[pipe_idx, 'diameter_m']

                G_temp = G_initial.copy()
                if G_temp.has_edge(from_junc, to_junc):
                    G_temp.remove_edge(from_junc, to_junc)
                    
                    still_connected_to_source = False
                    if nx.is_connected(G_temp):
                        if source_node in G_temp:
                           all_nodes_in_G_temp = list(G_temp.nodes())
                           still_connected_to_source = all(nx.has_path(G_temp, source_node, node) for node in all_nodes_in_G_temp)
                    
                    if still_connected_to_source:
                        try:
                            net.pipe.drop(index=pipe_idx, inplace=True)
                            pp.create_valve(
                                net,
                                from_junction=from_junc, 
                                to_junction=to_junc,   
                                diameter_m=diameter_val,
                                opened=False, 
                                name=f"Valve_on_original_pipe_{pipe_idx}"
                            )
                            G_initial.remove_edge(from_junc, to_junc) 
                            closed_valves_info.append((from_junc, to_junc))
                        except Exception as e:
                            logging.warning(f"Sim {net.name}: Could not replace pipe {pipe_idx} with valve: {e}. Skipping closure.")
    return net

## add_consumo_y_fugas

In [ ]:
def add_consumo_y_fugas(net):
    connected_nodes = set()
    if not net.pipe.empty:
        connected_nodes.update(net.pipe.from_junction.unique())
        connected_nodes.update(net.pipe.to_junction.unique())
    
    nodos_validos = [
        j for j in net.junction.index 
        if j != net.ext_grid.junction.iloc[0] and j in connected_nodes
    ]

    if not nodos_validos:
        logging.warning(f"Sim {net.name}: No valid connected nodes (excluding source) for consumptions/leaks.")
        return [], [] 
    
    consumos_info = []
    num_consumos_actual = 0
    if nodos_validos: 
        num_consumos_actual = min(len(nodos_validos), np.random.randint(*PARAMS['num_consumos']))

    nodos_para_consumo = []
    if num_consumos_actual > 0 : 
        nodos_para_consumo = np.random.choice(nodos_validos, size=num_consumos_actual, replace=False)
    
    for junc in nodos_para_consumo:
        consumo_val = np.random.uniform(*PARAMS['consumo_normal'])
        pp.create_sink(net, junc, mdot_kg_per_s=consumo_val, name=f"Sink_Consumo_{junc}")
        consumos_info.append({'node': junc, 'type': 'consumo', 'rate': consumo_val})

    G_temp = nx.Graph()
    if not net.pipe.empty:
         G_temp.add_edges_from(net.pipe[['from_junction', 'to_junction']].values)
    if hasattr(net, 'valve') and not net.valve.empty: 
        open_valves = net.valve[net.valve.opened] 
        if not open_valves.empty:
            G_temp.add_edges_from(open_valves[['from_junction', 'to_junction']].values)

    if not G_temp.nodes() or not nx.is_connected(G_temp): 
        logging.warning(f"Sim {net.name}: Graph G_temp (pipes + open valves) is empty or not connected. Skipping leak placement based on centrality.")
        return [c['node'] for c in consumos_info], [] 

    nodos_criticos_top = []
    try:
        potential_leak_nodes = [n for n in nodos_validos if n not in nodos_para_consumo and n in G_temp.nodes()]
        
        if potential_leak_nodes:
            betweenness = nx.betweenness_centrality(G_temp)
            nodos_criticos_candidatos = [n for n in potential_leak_nodes if n in betweenness]

            if not nodos_criticos_candidatos and potential_leak_nodes: 
                 nodos_criticos_candidatos = potential_leak_nodes 

            if nodos_criticos_candidatos:
                nodos_criticos_candidatos.sort(key=lambda n: betweenness.get(n, 0), reverse=True)
                nodos_criticos_top = nodos_criticos_candidatos[:min(15, len(nodos_criticos_candidatos))]

    except Exception as e: 
        logging.error(f"Sim {net.name}: Error selecting critical nodes for leaks: {e}. Using random valid nodes if available.")
        fallback_leak_nodes = [n for n in nodos_validos if n not in nodos_para_consumo and n in G_temp.nodes()]
        if fallback_leak_nodes:
            nodos_criticos_top = np.random.choice(
                fallback_leak_nodes,
                size=min(PARAMS['num_fugas'][1], len(fallback_leak_nodes)), 
                replace=False
            ).tolist()

    fugas_info = []
    if np.random.rand() < PARAMS['prob_fuga'] and nodos_criticos_top: 
        num_fugas_actual = min(len(nodos_criticos_top), np.random.randint(*PARAMS['num_fugas']))
        if num_fugas_actual > 0 : 
            nodos_para_fugas = np.random.choice(nodos_criticos_top, size=num_fugas_actual, replace=False)
            for junc in nodos_para_fugas:
                rate = np.random.uniform(*PARAMS['tasa_fuga'])
                pp.create_sink(net, junc, mdot_kg_per_s=rate, name=f"Sink_Fuga_{junc}")
                fugas_info.append({'node': junc, 'type': 'fuga', 'rate': rate})
    
    fugas_nodos = [f['node'] for f in fugas_info]
    consumos_nodos = [c['node'] for c in consumos_info]
    return fugas_nodos, consumos_nodos

## simulate_network

In [ ]:
def simulate_network(net):
    try:
        if net.ext_grid.empty:
            logging.error(f"[Sim_ID {net.name}] Error: No External Grid defined.")
            return None
        
        has_flow_elements = not net.pipe.empty
        if not has_flow_elements and hasattr(net, 'valve') and not net.valve.empty:
            if net.valve.opened.any(): 
                has_flow_elements = True
        
        if not has_flow_elements:
             logging.error(f"[Sim_ID {net.name}] Error: No pipes or open valves in the network for flow.")
             return None

        pp.pipeflow(
            net, friction_model="nikuradse", mode="hydraulics",
            tol_p=1e-3, tol_m=1e-3, max_iter=200
        )
        if net.converged:
            pp.pipeflow(
                net, friction_model="colebrook", mode="hydraulics",
                tol_p=1e-4, tol_m=1e-4, max_iter=300, max_iter_colebrook=150
            )
        
        if not net.converged:
            logging.warning(f"[Sim_ID {net.name}] Simulation did not converge.")
            return None
        return net

    except Exception as e:
        logging.error(f"[Sim_ID {net.name}] Error in simulation: {e}", exc_info=False) 
        return None

## calcular_caracteristicas

In [ ]:
def calcular_caracteristicas(net, fugas, consumos, time_step=0, sim_id=0):
    if not net or not getattr(net, 'converged', False) \
       or not hasattr(net, 'res_junction') or net.res_junction.empty:
        logging.warning(f"Sim_ID {sim_id} ({getattr(net, 'name','n/a')}): "
                        "No convergió o no hay resultados.")
        return pd.DataFrame()

    try:
        if 't_k' in net.res_junction.columns and not net.res_junction['t_k'].empty:
            temperature_K = net.res_junction['t_k'].mean()
        else:
            temperature_K = net.junction['tfluid_k'].mean()
        density_val = net.fluid.get_density(temperature_K)
    except Exception as e:
        logging.error(f"Sim_ID {sim_id} ({net.name}): Error al obtener densidad: {e}")
        density_val = 0.8

    if not net.res_pipe.empty:
        net.res_pipe['mdot_kg_per_s'] = 0.0
    if 'v_mean_m_per_s' in net.res_pipe.columns:
        mdot_list = []
        for idx, row in net.res_pipe.iterrows():
            if idx in net.pipe.index:
                d = net.pipe.at[idx, 'diameter_m']
                area = np.pi * (d/2)**2
                mdot_list.append(density_val * row['v_mean_m_per_s'] * area)
            else:
                mdot_list.append(0.0)
        net.res_pipe['mdot_kg_per_s'] = mdot_list

    G = nx.Graph()
    if not net.pipe.empty:
        G.add_edges_from(net.pipe[['from_junction','to_junction']].values)
    if hasattr(net, 'valve') and not net.valve.empty:
        ov = net.valve[net.valve.opened]
        if not ov.empty:
            G.add_edges_from(ov[['from_junction','to_junction']].values)

    source = net.ext_grid.junction.values[0] if not net.ext_grid.empty else None
    centrality, distances = {}, {}
    if source is not None and G.nodes:
        if source in G and nx.is_connected(G):
            centrality = nx.betweenness_centrality(G)
            distances = dict(nx.shortest_path_length(G, source))
        else:
            for comp in nx.connected_components(G):
                if source in comp:
                    sub = G.subgraph(comp)
                    centrality = nx.betweenness_centrality(sub)
                    distances = dict(nx.shortest_path_length(sub, source))
                    break

    features = []
    for j, row in net.res_junction.iterrows():
        pr = row['p_bar']
        ps = simulate_sensor_reading(pr, time_step)

        # flujos entrantes y salientes
        inflow = net.res_pipe.loc[
            net.pipe[net.pipe.to_junction==j].index, 'mdot_kg_per_s'
        ].abs().sum() if not net.pipe.empty else 0.0
        outflow = net.res_pipe.loc[
            net.pipe[net.pipe.from_junction==j].index, 'mdot_kg_per_s'
        ].abs().sum() if not net.pipe.empty else 0.0

        cons = net.sink.loc[net.sink.junction==j, 'mdot_kg_per_s'].sum() \
               if not net.sink.empty else 0.0
        delta_flow = abs(inflow - outflow - cons)

        neigh = list(G.neighbors(j)) if j in G else []
        pres_neigh = [net.res_junction.at[n,'p_bar'] for n in neigh if n in net.res_junction.index]
        d_pres = pr - np.mean(pres_neigh) if pres_neigh else 0.0
        std_pres = np.std(pres_neigh) if pres_neigh else 0.0

        z_local = 0.0
        if pres_neigh:
            arr = [pr] + pres_neigh
            if len(arr)>1 and np.std(arr)>1e-9:
                z_local = stats.zscore(arr)[0]

        features.append([
            sim_id, j, pr, ps, inflow, outflow, delta_flow,
            d_pres, centrality.get(j,0), distances.get(j,-1),
            len(neigh), 1 if j in fugas else 0,
            1 if j in consumos else 0, time_step%24,
            np.log(pr+1e-6), std_pres, z_local
        ])

    cols = [
        'sim_id','nodo','presion_real','presion_sensor',
        'flujo_entrada_tuberia','flujo_salida_tuberia','delta_flujo_nodo',
        'delta_presion_media_vecinos','centralidad','distancia_fuente',
        'grado_conectividad','fuga','es_consumo_discreto',
        'hora_dia','log_presion_real','std_presion_vecinos',
        'zscore_presion_local'
    ]
    return pd.DataFrame(features, columns=cols)

## generar_dataset_realista

In [ ]:
def generar_dataset_realista(num_muestras_por_hora=100, tiempo_max_horas=24, output_file="dataset_fugas_gas_simulacion.csv"):
    datos_acumulados = []
    simulaciones_exitosas = 0
    simulaciones_intentadas = 0
    
    try:
        for t_hora in tqdm(range(tiempo_max_horas), desc="Simulando Horas"):
            for i_muestra in range(num_muestras_por_hora):
                current_sim_id = simulaciones_intentadas
                simulaciones_intentadas += 1
                
                net = create_network(sim_id_for_name=str(current_sim_id))

                fugas_nodos, consumos_nodos = add_consumo_y_fugas(net)
                net_simulada = simulate_network(net) 
                
                if net_simulada and net_simulada.converged:
                    df_features = calcular_caracteristicas(net_simulada, fugas_nodos, consumos_nodos, t_hora, current_sim_id)
                    if not df_features.empty:
                        datos_acumulados.append(df_features)
                        simulaciones_exitosas += 1
            
            if (t_hora + 1) % 6 == 0 and datos_acumulados:
                logging.info(f"Saving partial data at hour {t_hora}...")
                temp_dataset = pd.concat(datos_acumulados, ignore_index=True)
                temp_dataset.to_csv(f"partial_{output_file}", index=False)

    except KeyboardInterrupt:
        logging.warning("Process interrupted by user (KeyboardInterrupt). Saving collected data...")
    except Exception as e: 
        logging.error(f"Critical error during dataset generation loop: {e}", exc_info=True)
    finally:
        logging.info(f"\nTotal simulations attempted: {simulaciones_intentadas}")
        logging.info(f"Successful simulations that generated data: {simulaciones_exitosas}")
        
        if datos_acumulados: 
            final_dataset = pd.concat(datos_acumulados, ignore_index=True)
            
            final_dataset.dropna(subset=['sim_id', 'nodo', 'presion_real'], inplace=True) 
            
            if not final_dataset.empty: 
                final_dataset = final_dataset.sample(frac=1).reset_index(drop=True) 
                logging.info(f"Final dataset contains {len(final_dataset)} rows.")
                try:
                    final_dataset.to_csv(output_file, index=False)
                    logging.info(f"Dataset successfully saved to {output_file}")
                except Exception as e_save:
                    logging.error(f"Failed to save the final dataset to {output_file}: {e_save}")
                return final_dataset
            else:
                logging.warning("Dataset became empty after essential NaN value removal.")
        
        logging.warning("No data was generated or could be saved.")
        return pd.DataFrame()

if __name__ == "__main__":
    logging.info("Starting realistic dataset generation...")
    dataset_generado = generar_dataset_realista(
        num_muestras_por_hora=100, 
        tiempo_max_horas=24,      
        output_file="dataset_fugas_gas_realista_v4.csv" 
    )
    
    if not dataset_generado.empty:
        logging.info("\n--- Generated Dataset Summary ---")
        logging.info(f"Total rows: {len(dataset_generado)}")
        logging.info(f"Unique simulation IDs: {dataset_generado['sim_id'].nunique()}")
        
        if 'fuga' in dataset_generado.columns:
            logging.info("\n'fuga' feature distribution (%):")
            logging.info(f"\n{dataset_generado.fuga.value_counts(normalize=True).mul(100).round(2)}") 
        
        if 'es_consumo_discreto' in dataset_generado.columns:
            logging.info("\n'es_consumo_discreto' feature distribution (%):")
            logging.info(f"\n{dataset_generado.es_consumo_discreto.value_counts(normalize=True).mul(100).round(2)}")
            
        logging.info("\nDataFrame Info:")
        dataset_generado.info()
        logging.info("\nFirst 5 rows of the dataset:")
        logging.info(f"\n{dataset_generado.head()}")
    else:
        logging.error("The generated dataset is empty. Please review logs for details.")
    logging.info("Dataset generation process finished.")

## VISUALITZACIÓ DE LA XARXA

In [ ]:
def plot_network(net, fugas=None, filepath="network.png", title="Estructura de la Red"):
    plt.figure(figsize=(12, 10))
    
    G = nx.from_pandas_edgelist(
        net.pipe,
        source='from_junction',
        target='to_junction',
        create_using=nx.DiGraph()
    )
    
    pos = nx.spring_layout(G, seed=42)
    
    fugas = fugas or []
    node_colors = ['red' if node in fugas else 'skyblue' for node in G.nodes()]
    node_sizes  = [200   if node in fugas else  50        for node in G.nodes()]
    
    nx.draw_networkx_nodes(G, pos,
                           node_color=node_colors,
                           node_size=node_sizes,
                           edgecolors='k',
                           linewidths=0.5)
    nx.draw_networkx_edges(G, pos,
                           arrowstyle='-',
                           arrowsize=8,
                           alpha=0.6)
    
    nx.draw_networkx_labels(G, pos, font_size=8)
    
    plt.scatter([], [], c='red',   s=50, label='Nodos con Fuga')
    plt.scatter([], [], c='skyblue', s=50, label='Nodos Normales')
    plt.legend(scatterpoints=1, frameon=True)
    
    plt.title(title)
    plt.axis('off')
    plt.tight_layout()
    plt.savefig(filepath, dpi=300)
    plt.close()
    print(f"Imagen guardada en {filepath}")

net = create_network(sim_id_for_name="ejemplo")
fugas, consumos = add_consumo_y_fugas(net)
net = simulate_network(net)

if net and net.converged:
    plot_network(net,
                 fugas=[],
                 filepath="red_sin_fugas.png",
                 title="Red de Gas SIN Fugas")
    
    plot_network(net,
                 fugas=fugas,
                 filepath="red_con_fugas.png",
                 title="Red de Gas CON Fugas")
else:
    print("La simulación no convergió; no se generaron imágenes.")
